In [1]:
import os
import pandas as pd
import re

**Loading and merging the data**

In [2]:
# Define the path to the directory containing the CSV files
# 'os.path.join' is used to construct a directory path by joining folder names
# The resulting path is relative to the current working directory
directory_path = os.path.join('..', 'data', 'kaggle_dataset', 'dataset_before_merge')

# List all CSV files in the directory
# 'os.listdir' lists all files and directories in 'directory_path'
# The list comprehension filters out only the files that end with '.csv'
# 'os.path.join(directory_path, file)' constructs the full file path for each CSV file
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Read the CSV files into DataFrames
# 'pd.read_csv(file)' reads each CSV file into a pandas DataFrame
# The resulting DataFrames are stored in a list
dataframes = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
# 'pd.concat(dataframes, ignore_index=True)' concatenates the list of DataFrames into one DataFrame
# 'ignore_index=True' resets the index in the resulting DataFrame
kaggle_dataset = pd.concat(dataframes, ignore_index=True)

**Exploring the data**

In [3]:
# Print the shape of the DataFrame 'kaggle_dataset'
# 'shape' returns a tuple representing the dimensionality of the DataFrame
# The first value is the number of rows, and the second value is the number of columns
print(kaggle_dataset.shape)

(460299, 3)


In [5]:
# Print the names of the columns in the DataFrame 'kaggle_dataset'
# 'columns' returns an Index object containing the column labels of the DataFrame
print(kaggle_dataset.columns)

Index(['owner', 'issue_title', 'description'], dtype='object')


In [6]:
# Display the DataFrame 'kaggle_dataset'
# This will print the first and last 5 rows of the DataFrame along with the column names and index
# Useful for a quick overview of the data after processing
kaggle_dataset

,owner,issue_title,description
0,amit@chromium.org,"Scrolling with some scroll mice (touchpad, etc...",Product Version : <see about:version>URLs...
1,jon@chromium.org,Proxy causes some or all network requests to fail,Product Version : 0.2.149.27 (1583)URLs (...
2,pfeldman@chromium.org,"Web inspector button ""dock to main window"" doe...",Product Version : chrome beta 1URLs (if a...
3,jon@chromium.org,Habari admin interface is not rendered correctly,Product Version : 0.2.149.27 (1583)URLs (...
4,pkasting@chromium.org,Maximize on second larger monitor not working,Product Version : 0.2.149.27URLs (if appl...
...,...,...,...
460294,navabi@chromium.org,Launch clank_qa recipes to the waterfall,We had git trouble
460295,bulach@chromium.org,data race in ThreadWatcherListTest,r255322 is culprithttp://build.chromium.org/p/...
460296,pfeldman@chromium.org,window.console object should not be configurable,Recently sites have begun replacing window.con...
460297,ernstm@chromium.org,Windows GPU bots failing on multiple tests,All Windows GPU bots are failing a variety of ...


In [16]:
# Print a concise summary of the DataFrame 'kaggle_dataset'
# 'info()' provides essential information about the DataFrame, including:
# - The number of non-null entries in each column
# - The data type of each column
# - The memory usage of the DataFrame
# This is useful for understanding the structure, completeness, and memory footprint of the data
print(kaggle_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460299 entries, 0 to 460298
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   owner        460299 non-null  object
 1   issue_title  460295 non-null  object
 2   description  460295 non-null  object
dtypes: object(3)
memory usage: 10.5+ MB
None


**Delete duplicate data [if exist]**

In [4]:
# Check if there are any duplicate rows in the DataFrame 'kaggle_dataset'
# 'duplicated()' returns a Series of boolean values indicating whether each row is a duplicate of a previous row
# 'any()' returns True if any element in the Series is True (indicating the presence of duplicates), otherwise False
# This is useful for determining if there are any duplicate rows in the DataFrame
kaggle_dataset.duplicated().any()

True

In [5]:
# Remove all duplicate rows from the DataFrame 'kaggle_dataset'
# 'drop_duplicates()' removes duplicate rows, keeping only the first occurrence of each set of duplicates
# The updated DataFrame without duplicates is assigned back to 'kaggle_dataset'
# This ensures that all rows in the DataFrame are unique
kaggle_dataset = kaggle_dataset.drop_duplicates()

In [6]:
# Check if there are any duplicate rows in the DataFrame 'kaggle_dataset'
# 'duplicated()' returns a Series of boolean values indicating whether each row is a duplicate of a previous row
# 'any()' returns True if any element in the Series is True (indicating the presence of duplicates), otherwise False
# This is useful for determining if there are any duplicate rows in the DataFrame
kaggle_dataset.duplicated().any()

False

In [7]:
# Print the shape of the DataFrame 'kaggle_dataset'
# 'shape' returns a tuple representing the dimensionality of the DataFrame
# The first value is the number of rows, and the second value is the number of columns
print(kaggle_dataset.shape)

(118587, 3)


**generates descriptive statistics of the data**

In [21]:
# generates descriptive statistics of the data
# include="O": This parameter is specifying that you want to include columns with object data type 
# By default, describe works on numeric columns
kaggle_dataset.describe(include = "O")

,owner,issue_title,description
count,118587,118586,118586
unique,2564,118012,116371
top,estade@chromium.org,Skia image rebaseline,See the link to graphs below.
freq,1249,38,1398


In [22]:
# Count the number of unique values in each column of the DataFrame 'kaggle_dataset'
# 'nunique()' returns a Series with the number of unique values for each column
# This is useful for understanding the variability and cardinality of data in each column
kaggle_dataset.nunique()

owner            2564
issue_title    118012
description    116371
dtype: int64

In [ ]:
'''
number of unique titles is less than number of rows
so there are duplicate titles
remove rows of duplicate titles
'''

**Remove duplicate titles**

In [8]:
# Remove duplicate rows from the DataFrame 'kaggle_dataset' based on the 'issue_title' column
# 'subset' specifies the column to consider when identifying duplicates
# 'drop_duplicates(subset='issue_title')' keeps the first occurrence of each unique 'issue_title' and removes the rest
# The updated DataFrame without duplicates is assigned back to 'kaggle_dataset'
# This ensures that each issue title appears only once in the DataFrame
kaggle_dataset = kaggle_dataset.drop_duplicates(subset='issue_title')

In [9]:
# Print the shape of the DataFrame 'kaggle_dataset'
# 'shape' returns a tuple representing the dimensionality of the DataFrame
# The first value is the number of rows, and the second value is the number of columns
print(kaggle_dataset.shape)

(118013, 3)


In [25]:
# Count the number of unique values in each column of the DataFrame 'kaggle_dataset'
# 'nunique()' returns a Series with the number of unique values for each column
# This is useful for understanding the variability and cardinality of data in each column
kaggle_dataset.nunique()

owner            2563
issue_title    118012
description    115812
dtype: int64

**generates descriptive statistics of the data**

In [26]:
# Generate summary statistics for columns with object dtype in the DataFrame 'kaggle_dataset'
# 'describe(include="O")' provides descriptive statistics for columns containing object-type (string) data
# The output includes count, unique, top, and freq for each object-type column
# This is useful for gaining insights into categorical or textual data in the DataFrame
kaggle_dataset.describe(include="O")

,owner,issue_title,description
count,118013,118012,118012
unique,2563,118012,115812
top,estade@chromium.org,"Scrolling with some scroll mice (touchpad, etc...",See the link to graphs below.
freq,1243,1,1396


**Show the number of nulls in each column**

In [27]:
# Print the sum of missing values (NaN) in each column of the DataFrame 'kaggle_dataset'
# 'isnull()' returns a DataFrame of boolean values indicating where NaN values are present
# 'sum()' then sums these boolean values along each column, giving the count of NaN values in each column
# This is useful for identifying columns with missing data in the DataFrame
print(kaggle_dataset.isnull().sum())

owner          0
issue_title    1
description    1
dtype: int64


**Remove rows with null values**

In [10]:
# Remove rows with missing values (NaN) from the DataFrame 'kaggle_dataset'
# 'dropna(inplace=True)' drops all rows containing NaN values from the DataFrame 'kaggle_dataset'
# 'inplace=True' modifies 'kaggle_dataset' directly without creating a new DataFrame
# This ensures that the DataFrame only contains rows with complete data
kaggle_dataset.dropna(inplace=True)
print(kaggle_dataset.isnull().sum())

owner          0
issue_title    0
description    0
dtype: int64


**Print the minimum number of occurance of owner column**

In [11]:
# Calculate the minimum number of occurrences of each unique value in the 'owner' column of the DataFrame 'kaggle_dataset'
# 'value_counts()' counts the occurrences of each unique value in the 'owner' column
# 'min()' then calculates the minimum number of occurrences among these values
min_occurrences = kaggle_dataset['owner'].value_counts().min()

# Print the minimum number of occurrences in the 'owner' column
print(f"The minimum number of occurrences in owner column is {min_occurrences}")

The minimum number of occurrences in owner column is 1


**Filter the dataset do the minimum occurance of each owner is 5**

In [13]:
# Calculate the occurrences of each value in the 'owner' column of the DataFrame 'kaggle_dataset'
value_counts = kaggle_dataset['owner'].value_counts()

# Filter 'kaggle_dataset' to include only rows where the 'owner' column has at least 5 occurrences
# 'value_counts[value_counts >= 5].index' filters the index (unique values) where the occurrence count is at least 5
# 'kaggle_dataset['owner'].isin(...)' filters rows based on whether their 'owner' value is in the filtered index
kaggle_dataset = kaggle_dataset[kaggle_dataset['owner'].isin(value_counts[value_counts >= 5].index)]

In [14]:
# Calculate the minimum number of occurrences of each unique value in the 'owner' column of the DataFrame 'kaggle_dataset'
# 'value_counts()' counts the occurrences of each unique value in the 'owner' column
# 'min()' then calculates the minimum number of occurrences among these values
min_occurrences = kaggle_dataset['owner'].value_counts().min()

# Print the minimum number of occurrences in the 'owner' column
print(f"The minimum number of occurrences in owner column is {min_occurrences}")

The minimum number of occurrences in owner column after filteration is 5


In [15]:
# Print the shape of the DataFrame 'kaggle_dataset'
# 'shape' returns a tuple representing the dimensionality of the DataFrame
# The first value is the number of rows, and the second value is the number of columns
print(kaggle_dataset.shape)

(116298, 3)


In [16]:
# Count the number of unique values in each column of the DataFrame 'kaggle_dataset'
# 'nunique()' returns a Series with the number of unique values for each column
# This is useful for understanding the variability and cardinality of data in each column
kaggle_dataset.nunique()

owner            1688
issue_title    116298
description    114125
dtype: int64

**Merge the title and description into one (Summary) column**

In [17]:
def preprocess_summary(summary):
    """
    Preprocesses a summary string by removing special characters, newlines, and hyperlinks.

    Parameters:
    summary (str): The summary string to preprocess.

    Returns:
    str: The preprocessed summary string.
    """
    # Remove special characters using regular expression
    summary = re.sub(r'[^\w\s]', '', summary)
    
    # Remove newlines by replacing with a space
    summary = summary.replace('\n', ' ')
    
    # Remove hyperlinks using regular expression
    summary = re.sub(r'http\S+', '', summary)
    
    return summary

In [18]:
def filter_by_word_count(df, min_word_count):
    """
    Filter a DataFrame 'df' to include only rows where the 'Summary' column has at least 'min_word_count' words.

    Parameters:
    df (DataFrame): The input DataFrame containing a 'Summary' column.
    min_word_count (int): The minimum number of words required in the 'Summary' column.

    Returns:
    DataFrame: A filtered DataFrame containing rows with at least 'min_word_count' words in the 'Summary' column.
    """
    # Split each summary into words, count the number of words, and filter rows based on word count
    return df[df['Summary'].str.split().str.len() >= min_word_count]

In [19]:
# This assigns the values from 'issue_title' column to the 'Summary' column
kaggle_dataset['Summary'] = kaggle_dataset['issue_title']

In [20]:
# Preprocess the 'Summary' column in the DataFrame 'kaggle_dataset'
# 'apply(preprocess_summary)' applies the function 'preprocess_summary' to each element in the 'Summary' column
# This preprocesses each summary by removing special characters, newlines, and hyperlinks
kaggle_dataset['Summary'] = kaggle_dataset['Summary'].apply(preprocess_summary)

In [21]:
# Drop the 'issue_title' and 'description' columns from the DataFrame 'kaggle_dataset'
# 'drop(columns=['issue_title','description'])' removes the specified columns from the DataFrame
# The updated DataFrame without 'issue_title' and 'description' columns is assigned back to 'kaggle_dataset'
kaggle_dataset = kaggle_dataset.drop(columns=['issue_title', 'description'])

**Filter the dataset by the number of words in Summary column**

In [22]:
# Define the minimum word count required in each summary
min_word_count = 5

# Filter rows in the DataFrame 'kaggle_dataset' to include only those where the 'Summary' column has at least 'min_word_count' words
# 'filter_by_word_count(kaggle_dataset, min_word_count)' filters the DataFrame based on the word count in the 'Summary' column
kaggle_dataset = filter_by_word_count(kaggle_dataset, min_word_count)

In [23]:
# Print the shape of the DataFrame 'kaggle_dataset'
# 'shape' returns a tuple representing the dimensionality of the DataFrame
# The first value is the number of rows, and the second value is the number of columns
print(kaggle_dataset.shape)

(100047, 2)


In [24]:
# Print the names of the columns in the DataFrame 'kaggle_dataset'
# 'columns' returns an Index object containing the column labels of the DataFrame
print(kaggle_dataset.columns)

Index(['owner', 'Summary'], dtype='object')


**Save cleaned data**

In [25]:
# Get the current working directory
current_dir = os.getcwd()

# Define the relative path to save the cleaned DataFrame as a CSV file
relative_path = os.path.join('..', 'data', 'kaggle_dataset', 'cleaned_kaggle_dataset.csv')

# Save the DataFrame 'kaggle_dataset' to a CSV file at the specified path
# 'index=False' excludes the DataFrame index from being saved to the CSV file
kaggle_dataset.to_csv(os.path.join(current_dir, relative_path), index=False)